# Kuramoto-Sivashinsky Equation

$$
u_t = -u u_x - u_{xx} - \nu u_{xxxx}  \qquad (0 < x < L)
$$

$$
u_t = -\lambda u u_x - \lambda^2 u_{xx} - \nu \lambda^4 u_{xxxx}, \qquad (0 < x < 1)
$$

## Importing modules 

In [1]:
import time

import numpy as np
import equations as eq

import h5py

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
from IPython.display import HTML

## Setting Parameters 

In [2]:
NW = 48
ks01 = eq.KS(NW)
args = ks01.getParamDefault()

# N2 = ks01.N2
J = ks01.J
x = ks01.sc.get_x()

## Setting initial data 

In [3]:
u0c = ks01.sc.get_one_cwave((1,)) * 0.5

## Computation 

In [4]:
trange0 = np.arange(0., 50.0, 2.0)
trange1 = np.arange(0., 450.0, 2.0)

max_step = 1.0e-1

In [5]:
fname = 'ks.hdf5'

### Comparison of Several Solvers

See [the official document](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html "scipy doc")
for the information of available solvers.

In [6]:
with h5py.File(fname, 'w') as fh:
    ks01.mkInitDataSet(u0c, fh)
    ks01.evolve(fh, trange0, tuple(args), NTlump=5,
                max_step=max_step, method='RK23',
                pb_type="notebook"
               )

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
with h5py.File(fname, 'w') as fh:
    ks01.mkInitDataSet(u0c, fh)
    ks01.evolve(fh, trange0, tuple(args), NTlump=5, 
                max_step=max_step, method='Radau',
                pb_type="notebook"
               )

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
with h5py.File(fname, 'w') as fh:
    ks01.mkInitDataSet(u0c, fh)
    ks01.evolve(fh, trange0, tuple(args), NTlump=5,
                max_step=max_step, method='BDF',
                pb_type="notebook"
               )

  0%|          | 0/5 [00:00<?, ?it/s]

### Continuation of the Computation

In [9]:
with h5py.File(fname, 'r+') as fh:
    ks01.evolve(fh, trange1, tuple(args), 
                max_step=max_step, method='Radau',
                pb_type="notebook"
               )

  0%|          | 0/2 [00:00<?, ?it/s]

## Visualization

In [10]:
with h5py.File(fname, 'r') as fh:
    wp = fh['wp'][()]
    time = fh['trange'][()]

In [11]:
gsP = GridSpec(1,1)
gsT = GridSpec(1,1)
gsP.update(bottom=0.3, top=0.9)
gsT.update(bottom=0.05, top=0.2)
fig = plt.figure(figsize=(4,5))

ax = fig.add_subplot(gsP[0])
ax.set_ylim((-3.0, 3.0))
ax.set_xlim((-0.1, 1.1))
ax.grid(True)
lines = plt.plot(x, [np.nan]*len(x))

timeframe = fig.add_subplot(gsT[0])
timeframe.set_axis_off()
timer = timeframe.text(0.5,0.5,
                       '', 
                       va='center', ha="center"
                      )
timeformat = "time = {0:08.3f}"

plt.close()

step = 20

def init():
    for l in lines:
        l.set_ydata([np.nan] * len(x))
    timer.set_text(timeformat.format(time[0]))
    return lines + [timer]

def animate(i):
    idx = i*step
    for l in lines:
        l.set_ydata(wp[idx,:])
    timer.set_text(timeformat.format(time[idx]))
    return lines + [timer]

ani = animation.FuncAnimation(
    fig, animate, frames=wp.shape[0] // step,
    init_func=init, interval=100, blit=True,
    repeat=False)

# plt.show()

HTML(ani.to_jshtml())